## Modules

In [255]:
import numpy as np
import pandas as pd
from itertools import chain

## Functions

In [256]:
def crear_individuo(num_genes=20):
    return [int(x>0.5) for x in np.random.uniform(size=num_genes)]

In [257]:
def objetivo(x):
    return sum([int(g) for g in x])

In [258]:
def seleccion(objetivo,pob,k = 3):
    pob = pd.DataFrame({'x':pob}).sample(k)
    pob['aptitud'] = pob['x'].map(lambda x:objetivo(x))
    pob.sort_values(by='aptitud',inplace=True)
    return pob.tail(1)['x'].values[0]

In [259]:
def combinacion(padre1,padre2,proba):
    #Los hijos son por defecto copias de los padres
    padre1 = [g for g in padre1]
    padre2 = [g for g in padre2]
    hijos = [padre1,padre2]
    if np.random.uniform()<proba:
        #Seleccionar un punto de combinacion que no esté
        #en los extremos
        i = np.random.choice(range(1,len(padre1)-1))
        hijos = padre1[:i]+padre2[i:],padre2[:i]+padre1[i:]
    return ["".join([str(x) for x in h]) for h in hijos]

In [260]:
def mutacion(individuo,proba=0.5):
    return "".join([str(1-int(g)) if r<proba else g for g,r in zip(individuo,np.random.uniform(size=len(individuo)))])

In [261]:
def alg_genetico(objetivo,n_genes=20,n_generaciones=100,tam_pob=100,proba_comb=0.9,proba_mut=1/20,tam_muestra=3):

    pob = [crear_individuo(num_genes=n_genes) for _ in range(tam_pob)]
    for gen in range(n_generaciones):
        aptos = [seleccion(objetivo,pob,k=tam_muestra) for _ in range(tam_pob)]
        hijos = [combinacion(*aptos[i:i+2],proba_comb) for i in range(0,tam_pob,2)]
        hijos = list(map(lambda x: mutacion(x,proba=proba_mut),chain(*hijos)))
        pob = hijos[:]
    pob = pd.DataFrame({'x':pob})
    pob['aptitud'] = pob['x'].map(objetivo)
    pob.sort_values(by='aptitud',inplace=True)
    return pob.tail(1).values.tolist()[0]

In [262]:
def decodificar(x):
    x="".join([str(s) for s in x])
    algoritmo= int(x[:2],2)
    if(algoritmo==3):
        algoritmo=2
    penalizacion=int(x[2:4],2)
    if(penalizacion==3):
        penalizacion=2
    max_iter= int(x[9:],2)
    int_num = int(x[4:6], 2)
    frac_num = sum(int(bit) * 2 ** -(i + 1) for i, bit in enumerate(x[6:9]))
    lr= int_num+frac_num
    return algoritmo,penalizacion,max_iter,lr

In [263]:
def Z(x,y):
    return np.exp(-1/3*x**3+x-y**2)

## Testing functions

In [264]:
alg_genetico(objetivo)

['11111111111111111111', 20]